# Diluted Unitary Fitting, Three Qubits

Fitting Diluted Unitaries on the spectra of [repeated quantum ciruits](../experiments/Map_Retrival_Repeated_PQC_3qubits.ipynb).

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from quantum_tools import generate_unitary
from quantum_channel import channel_spectrum
from loss_functions import SpectrumDistance
from optimization import ModelQuantumMap, Logger
from kraus_channels import KrausMap, DilutedKrausMap
from utils import saver
from spectrum import mean_spacing
from utils import loader

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
def optimize_spectral_distance(spectrum_target = None, 
                              d = None, 
                              c_start = 0.5,
                              rank_start = 1,
                              rank_step = 1,
                              num_iter = 1000,
                              lr = 0.001,
                              verbose = True): 
    
    model_list = []
    distance_list = []
    counter = 0

    sigma = mean_spacing(spectrum_target)
    
    for rank in range(rank_start, d**2+rank_step, rank_step): 
        model = ModelQuantumMap(channel = DilutedKrausMap(U=generate_unitary(d), 
                                                          c=c_start, 
                                                          kraus_part=KrausMap(d, 
                                                                              rank=rank, 
                                                                              trainable=False),
                                                     ),
                            loss_function = SpectrumDistance(sigma=sigma, k = None, remove_shift=True),
                            optimizer = tf.optimizers.Adam(learning_rate=lr),
                            logger = Logger(loss_function = SpectrumDistance(sigma=sigma, k = None), 
                                            verbose=verbose,
                                            N=0),
                            )
        model.train(inputs = None,
                    targets = [spectrum_target],
                    num_iter = num_iter,
                    verbose = verbose,
                   )

        c_start = model.channel.c.numpy()
        model_list.append(model)
        distance_list.append(model.logger.loss_train_list[-1])
        
        idx = np.argmin(distance_list)
        if distance_list[-1] > distance_list[idx]:
            counter += 1
        else:
            counter = 0
            
        if counter > 3:
            print("Done!")
            break

    distance_best = distance_list[idx]
    model_best = model_list[idx]

    return distance_best, model_best

def fit_diluted_unitary(spectrum_target_list, 
                        d, 
                        rank_start, 
                        rank_step, 
                        num_iter = 1000,
                        lr = 0.001):
    
    distance_best_list = []
    model_best_list = []
    for spectrum_target in tqdm(spectrum_target_list):
        distance_best, model_best = optimize_spectral_distance(spectrum_target,
                                                              d,
                                                              rank_start,
                                                              rank_step,
                                                              num_iter = num_iter,
                                                              lr = lr)
        distance_best_list.append(distance_best)
        model_best_list.append(model_best)
    
    for model in model_best_list:
        model.optimizer = None
        
    return distance_best_list, model_best_list

## Three Qubits, Repetition

In [3]:
n = 4
d = 2**n

path1 = "../experiments/data/"
path2 = "data/"


[channel_list1, channel_list2, channel_list3, channel_list4, channel_list5, channel_list6, channel_list7] = loader(path1 + "channel_fullRank_belem_longTrain.p")

spectrum_target_list1 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list1]
spectrum_target_list2 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list2]
spectrum_target_list3 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list3]
spectrum_target_list4 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list4]
spectrum_target_list5 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list5]

In [ ]:
distance_best_list = []
model_best_list = []
for i, spectrum_target in enumerate(spectrum_target_list1):
    distance_best, model_best = optimize_spectral_distance(spectrum_target = spectrum_target, 
                                                        d = d,
                                                        c_start = 0.5,
                                                        rank_start = 1, 
                                                        rank_step = 1,
                                                        num_iter = 200,
                                                        lr=0.01,
                                                        verbose=True)

    model_best.optimizer = None
    distance_best_list.append(distance_best)
    model_best_list.append(model_best)

saver([distance_best_list, model_best_list], path2 + f"diluted_fitting_KL_3qubit_8layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

0.060713153762869905 None


KeyboardInterrupt: 

In [ ]:
distance_best_list = []
model_best_list = []
for i, spectrum_target in enumerate(spectrum_target_list1):
    distance_best, model_best = optimize_spectral_distance(spectrum_target = spectrum_target, 
                                                        d = d,
                                                        c_start = 0.5,
                                                        rank_start = 1, 
                                                        rank_step = 1,
                                                        num_iter = 200,
                                                        lr=0.01,
                                                        verbose=True)

    model_best.optimizer = None
    distance_best_list.append(distance_best)
    model_best_list.append(model_best)

saver([distance_best_list, model_best_list], path2 + f"diluted_fitting_KL_3qubit_8layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

0.04533811068121134 None
0.045329996617866984 None
0.0453299964368201 None


  0%|          | 0/200 [00:00<?, ?it/s]

0.048500407705357323 None
-0.004775618224579954 None
-0.0050628142558500275 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.024810470474832036 None
-0.024823852118013703 None
-0.02482385386643443 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.02270932018871931 None
-0.023362337724488737 None
-0.023362361145985916 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.022285456585814212 None
-0.022615400341474218 None
-0.022615407816214343 None
Done!


In [ ]:
distance_best_list = []
model_best_list = []
for i, spectrum_target in enumerate(spectrum_target_list1):
    distance_best, model_best = optimize_spectral_distance(spectrum_target = spectrum_target, 
                                                        d = d,
                                                        c_start = 0.5,
                                                        rank_start = 1, 
                                                        rank_step = 1,
                                                        num_iter = 200,
                                                        lr=0.01,
                                                        verbose=True)

    model_best.optimizer = None
    distance_best_list.append(distance_best)
    model_best_list.append(model_best)

saver([distance_best_list, model_best_list], path2 + f"diluted_fitting_KL_3qubit_8layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

-0.009984272049604045 None
-0.015264161447782622 None
-0.015264171334487528 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.013456041423519222 None
-0.015784388754267388 None
-0.01578442147142492 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.024435826185256602 None
-0.024453200951644452 None
-0.024453204241121615 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.02265450407977239 None
-0.022681747009569493 None
-0.02268174712055275 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.021485065906558506 None
-0.02296371545447011 None
-0.02296383567084083 None
Done!


In [ ]:
distance_best_list = []
model_best_list = []
for i, spectrum_target in enumerate(spectrum_target_list1):
    distance_best, model_best = optimize_spectral_distance(spectrum_target = spectrum_target, 
                                                        d = d,
                                                        c_start = 0.5,
                                                        rank_start = 1, 
                                                        rank_step = 1,
                                                        num_iter = 200,
                                                        lr=0.01,
                                                        verbose=True)

    model_best.optimizer = None
    distance_best_list.append(distance_best)
    model_best_list.append(model_best)

saver([distance_best_list, model_best_list], path2 + f"diluted_fitting_KL_3qubit_8layer.model")

  0%|          | 0/300 [00:00<?, ?it/s]

-0.015694027658555106 None
-0.02078156838127121 None
-0.020781693172220658 None
-0.020781693173686784 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.017454795201010502 None
-0.024090018006315467 None
-0.02409003538422623 None
-0.024090035390411102 None


  0%|          | 0/300 [00:00<?, ?it/s]

...

-0.02693588025331195 None
-0.02755070384587078 None
-0.027550709464451893 None
-0.027550709464918485 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.028289781671778744 None
-0.02921309510784325 None
-0.029213097432545312 None
-0.029213097433341946 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.027043310803115117 None
-0.027149640143401307 None
-0.027149640205306066 None
-0.02714964020533793 None
Done!


In [7]:
distance_best_list = []
model_best_list = []
for i, spectrum_target in enumerate(spectrum_target_list5):
    distance_best, model_best = optimize_spectral_distance(spectrum_target = spectrum_target, 
                                                        d = d,
                                                        c_start = 0.23,
                                                        rank_start = 33, 
                                                        rank_step = 1,
                                                        num_iter = 200,
                                                        lr=0.01,
                                                        verbose=True)

    model_best.optimizer = None
    distance_best_list.append(distance_best)
    model_best_list.append(model_best)

saver([distance_best_list, model_best_list], path2 + f"diluted_fitting_3qubit_40layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

0.019803784107841574 None
-0.10484649208319805 None
-0.1048594986669817 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.10043596207555489 None
-0.10608259440723125 None
-0.10608287207918883 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.12363409873816927 None
-0.12380639826834428 None
-0.12380640534255402 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.12268772646189222 None
-0.12270437668009285 None
-0.12270437797350331 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.12125466764456819 None
-0.12129613565505806 None
-0.12129613898316907 None
Done!
